In [ ]:
#Installing required packages

!pip install git+https://github.com/huggingface/transformers
!pip install git+https://github.com/huggingface/peft.git
!pip install torch
!pip install -q bitsandbytes accelerate

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-wugqp10l
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-wugqp10l
  Resolved https://github.com/huggingface/transformers to commit 43bfd093e1817c0333a1e10fcbdd54f1032baad0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.9 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.35.0.dev0-py3-none-any.whl size=7748680 sha256=6610935cc7e1bb1240a0ce91ddce9592860378fadcf7f58716c4ce3bfd811fd2
  Stored in directory: /tmp/pip-ephem-wheel-cache-tuuk0eo

In [ ]:
########## IN CASE OF BELOW ERROR WHEN INSTALLING PACKAGES USING PIP, RUN THIS CELL #####################

#---NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968---#

import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
#Importing libraries

from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import re

In [ ]:
#Loading adapter model and merging it with base model for inferencing

torch.set_default_device('cuda')

peft_model_id = "abhishek7/Prompt_diffusion-v0.1"

config = PeftConfig.from_pretrained(peft_model_id)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    low_cpu_mem_usage=True,
    load_in_4bit=True,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map="auto"
)

model = PeftModel.from_pretrained(model, peft_model_id)

model = model.merge_and_unload()

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:207: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Function to truncate text based on punctuation count

def truncate_text(text, max_punctuation):
    punctuation_count = 0
    truncated_text = ""
    for char in text:
        truncated_text += char
        if char in [',', '.']:
            punctuation_count += 1
            if punctuation_count >= max_punctuation:
                break
    # Replace the last comma with a full stop if the last punctuation is a comma
    if truncated_text.rstrip()[-1] == ',':
        truncated_text = truncated_text.rstrip()[:-1] + '.'

    return truncated_text


# Function to generate prompt

def generate_prompt(input, max_length, temperature):
    input_context = f'''
###Human:
generate a stable diffusion prompt for {input}

###Assistant:
'''

    inputs = tokenizer.encode(input_context, return_tensors="pt")
    outputs = model.generate(inputs, max_length=max_length, temperature=temperature, num_return_sequences=1, do_sample=True)

    output_text = tokenizer.decode(outputs[0], skip_special_tokens = True)

    # Extract the Assistant's response using regex
    match = re.search(r'###Assistant:(.*?)(###Human:|$)', output_text, re.DOTALL)
    if match:
        assistant_response = match.group(1)
    else:
        raise ValueError("No Assistant response found")



    # Truncate the Assistant's response based on the criteria, the output is truncated based on number of comma/full-stops.
    truncated_response = truncate_text(assistant_response, max_punctuation=10)

    return truncated_response

In [ ]:
# Usage:

input_text = "a beautiful woman dancing in rain"

prompt = generate_prompt(input_text, 150, 1)

print("\nPrompt: " + prompt)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:363: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Prompt: 
Beautiful woman dancing in the rain, with the water falling around her in soft, ethereal patterns. Her hair is loose and flowing, and her dress is translucent, revealing her figure as she moves. The background is blurred, focusing on her and the rain.


